In [ ]:
#default_exp data.load

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.notebook.showdoc import show_doc

In [ ]:
#export
from torch.utils.data.dataloader import _MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter,_DatasetKind
_loaders = (_MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter)

## DataLoader

In [ ]:
class SleepyDS():
    def __init__(self,coll): self.coll,self.rng = coll,random.Random()
    def __len__(self): return len(self.coll)
    def __getitem__(self,i):
        time.sleep(self.rng.random()/100)
        return self.coll[i]

def twoepochs(d): return ' '.join(''.join(o) for _ in range(2) for o in d)

testds = SleepyDS(string.ascii_lowercase)    
bs = 4

- set bs,drop_last,sampler after init
- collate_fn,kind,sampler,auto_collate from ds
  - auto_collate replaced by 
- figure ds type from attr, not inheritance
- transforms and reset
- define appropriate init params with subclass params, not bool chks

In [ ]:
def delegate_attr(k, o, to):
    if k.startswith('_') or k==to: raise AttributeError(k)
    try: return getattr(getattr(o,to), k)
    except AttributeError: raise AttributeError(k) from None

In [ ]:
#export
def _wif(worker_id):
    info = get_worker_info()
    ds = info.dataset
    ds.nw,ds.offs = info.num_workers,info.id
    ds.wif()

In [ ]:
#     def mk_sampler(self, **kwargs):
#         if not batch_size or batch_sampler: assert not shuffle and sampler and not drop_last
#         if batch_sampler: return batch_sampler
#         if sampler: assert not shuffle, "Can't shuffle a custom sampler"
#         else: sampler = (itertools.count() if self.is_iterable
#                          else (SequentialSampler,RandomSampler)[shuffle](self))
#         return BatchSampler(sampler, batch_size, drop_last)

In [ ]:
class Sampler:
    def __init__(self, items, drop_last=False): self.items,self.drop_last = items,drop_last
    def sample(self,b): return None
    def __len__(self): return len(self.items) - (1 if self.drop_last else 0)
    def __iter__(self):
        try: n = range_of(self)
        except: n = itertools.count()
        return map(self.sample, n)

In [ ]:
#export
class BaseDataset():
    _methods = 'collate_fn indexes batches reset wif'
    @kwargs(_methods, keep=True)
    def __init__(self, items=None, **kwargs):
        self.items = items
        for k in [k for k in kwargs if k in self._methods.split()]:
            setattr(self, k, types.MethodType(kwargs.pop(k),self))
        self.rng,self.sampler = random.Random(),self.mk_sampler(**kwargs)

    def __iter__(self):
        torch.manual_seed(self.rng.randint(0,sys.maxsize))
        self.reset()
        return map(self.collate_fn, self.batches())
    
    @classmethod
    def create(cls, items, **kwargs): return IndexedDataset(items, **kwargs)
    
    def __len__(self): return len(self.sampler)
    def mk_sampler(self, **kwargs): return Sampler(self.items, **kwargs)
    def collate_fn(self, b): return default_convert(b)
    def batch(self, s):
        if not is_iter(s): return self.item(s)
        res = []
        try:
            for s_ in s: res.append(self.item(s_))
        except StopIteration as e:
            if res: self.done=True
            else: raise e
        return res

    def item(self, s): return next(self.it)
    def wif(self): pass
    def reset(self):
        self.done=False
        if self.items: self.it = iter(self.items)

    def batches(self):
        try:
            for s in self.sampler:
                yield self.batch(s)
                if self.done: raise StopIteration
        except StopIteration:
            self.done=False
            return

Override `batches` to return some specific finite iterable.

In [ ]:
class LettersDS(BaseDataset):
    def batches(self): return (string.ascii_lowercase[i:i+4] for i in range(0,26,4))

test_eq(L(LettersDS()), 'abcd,efgh,ijkl,mnop,qrst,uvwx,yz'.split(','))

Override `batch` and use the default infinite sampler to get a stream of unknown length (`raise StopIteration` when you want to stop the stream).

In [ ]:
class RandDS(BaseDataset):
    def batch(self, s):
        r = random.random()
        if r>0.99: raise StopIteration
        return r

L(RandDS())

(#246) [0.4234170261400705,0.047414301940740144,0.8286873938234129,0.04176993365912274,0.03795914841374515,0.24209004391775046,0.17844070604387674,0.04797346720169737,0.03241544200208779,0.13923316033964017...]

`items` is assumed to have a `__next__` that returns a batch.

In [ ]:
ds1 = BaseDataset(testds)
test_eq(''.join(ds1), string.ascii_lowercase)
test_eq(len(ds1), 26)

t2 = L(tensor([0,1,2]),tensor([3,4,5]))
ds2 = BaseDataset(t2)
test_eq(list(ds2), t2)

t3 = L(array([0,1,2]),array([3,4,5]))
ds3 = BaseDataset(t3)
test_eq_type(L(ds3), t2)

ds4 = BaseDataset(t3, collate_fn=noops)
test_eq_type(L(ds4), t3)

In [ ]:
@delegates()
class BatchIterSampler(Sampler):
    def __init__(self, items, bs, **kwargs):
        super().__init__(items, **kwargs)
        self.bs=bs
    def sample(self,b): return [None]*self.bs
    def __len__(self): return (len(self.items)-1)//self.bs + (0 if self.drop_last else 1)

In [ ]:
s = BatchIterSampler([1,2,3,4,5,6,7],2)
L(s),len(s)

((#4) [[None, None],[None, None],[None, None],[None, None]], 4)

In [ ]:
s = BatchIterSampler(map(noop,range(8)),2)
it = iter(s)
test_fail(lambda:len(s))
L(next(it) for i in range(4))

(#4) [[None, None],[None, None],[None, None],[None, None]]

In [ ]:
#export
class BaseBatchDataset(BaseDataset):
    def __init__(self, items=None, bs=4, **kwargs): super().__init__(items, bs=bs, **kwargs)
    def mk_sampler(self, bs, **kwargs): return BatchIterSampler(self.items, bs, **kwargs)
    def collate_fn(self, b): return default_collate(b)

In [ ]:
ds1 = BaseBatchDataset(testds,drop_last=True)
twoepochs(ds1)

'abcd efgh ijkl mnop qrst uvwx abcd efgh ijkl mnop qrst uvwx'

In [ ]:
ds1 = BaseBatchDataset(range(20))
L(ds1)

(#5) [tensor([0, 1, 2, 3]),tensor([4, 5, 6, 7]),tensor([ 8,  9, 10, 11]),tensor([12, 13, 14, 15]),tensor([16, 17, 18, 19])]

In [ ]:
ds1 = BaseBatchDataset([str(i) for i in range(20)])
L(ds1)

(#5) [['0', '1', '2', '3'],['4', '5', '6', '7'],['8', '9', '10', '11'],['12', '13', '14', '15'],['16', '17', '18', '19']]

In [ ]:
class RandBatchDS(BaseBatchDataset):
    def item(self, s):
        r = random.random()
        if r>0.9: raise StopIteration
        return r

ds = RandBatchDS()
L(ds)

(#2) [tensor([0.2554, 0.4523, 0.5147, 0.5368], dtype=torch.float64),tensor([0.0519], dtype=torch.float64)]

In [ ]:
#export
class DataLoader:
    _auto_collation,collate_fn,drop_last,dataset_kind = False,noops,False,_DatasetKind.Iterable
    def __init__(self, dataset, num_workers=0, pin_memory=False, timeout=0, tfm=noop, **kwargs):
        self.dataset = dataset if isinstance(dataset, BaseDataset) else BaseDataset.create(dataset, **kwargs) 
        self.pin_memory,self.tfm,self.worker_init_fn = pin_memory,tfm,_wif
        self._index_sampler = self.dataset.sampler
        self.num_workers = 0 if num_workers < 0 else num_workers
        self.timeout = 0 if timeout < 0 else timeout

    def __iter__(self):  return map(self.tfm, _loaders[self.num_workers==0](self))
    def __getattr__(self,k): return delegate_attr(k,self,'dataset')
    def __len__(self): return len(self.dataset)

In [ ]:
for _ in range(4): print(L(DataLoader(ds)))

(#4) [tensor([0.7587, 0.7087, 0.0985, 0.7834], dtype=torch.float64),tensor([0.7923, 0.6998, 0.4943, 0.6190], dtype=torch.float64),tensor([0.6839, 0.6491, 0.2545, 0.2768], dtype=torch.float64),tensor([0.4087, 0.2419, 0.4599], dtype=torch.float64)]
(#3) [tensor([0.1546, 0.8043, 0.7304, 0.1451], dtype=torch.float64),tensor([0.3036, 0.5096, 0.4649, 0.5221], dtype=torch.float64),tensor([0.6010], dtype=torch.float64)]
(#1) [tensor([0.6590], dtype=torch.float64)]
(#2) [tensor([0.0174, 0.3323, 0.8283, 0.0660], dtype=torch.float64),tensor([0.1942, 0.2704, 0.5890], dtype=torch.float64)]


In [ ]:
for _ in range(4): print(L(DataLoader(ds, num_workers=4)))

(#12) [tensor([0.3528, 0.1082, 0.4106, 0.7953], dtype=torch.float64),tensor([0.6206, 0.6652, 0.0623, 0.0666], dtype=torch.float64),tensor([0.8420, 0.3056, 0.5540, 0.3665], dtype=torch.float64),tensor([0.2523, 0.7311, 0.6612, 0.6554], dtype=torch.float64),tensor([0.7499, 0.8144], dtype=torch.float64),tensor([0.8189, 0.0061, 0.1110, 0.6277], dtype=torch.float64),tensor([0.3307, 0.7486], dtype=torch.float64),tensor([0.4993, 0.3224, 0.3576, 0.5865], dtype=torch.float64),tensor([0.6720, 0.3018, 0.6998, 0.4193], dtype=torch.float64),tensor([0.4031, 0.7388], dtype=torch.float64)...]
(#15) [tensor([0.1093, 0.2307, 0.4622, 0.1012], dtype=torch.float64),tensor([0.7648, 0.0663, 0.3023, 0.0639], dtype=torch.float64),tensor([0.5816, 0.8246, 0.8726, 0.2368], dtype=torch.float64),tensor([0.7233, 0.7650, 0.0440, 0.4029], dtype=torch.float64),tensor([0.7295, 0.3058, 0.8589, 0.6006], dtype=torch.float64),tensor([0.0587, 0.2197, 0.1190, 0.0081], dtype=torch.float64),tensor([0.1781, 0.5315, 0.6847, 0.0188

In [ ]:
# Incomplete below

In [ ]:
test_eq(''.join(DataLoader(ds1, num_workers=0)), string.ascii_lowercase)
test_eq(L(DataLoader(ds2, num_workers=1)), t2)
# n workers means n copies of the iter, in some arbitrary order
test_eq(L(DataLoader(ds4, num_workers=2)).mapped(list).sorted(), (t3*2).mapped(list).sorted())

In [ ]:
# def mk_collate_fn(auto_collation): return (default_convert,default_collate)[auto_collation]

In [ ]:
#export
class IndexedDataset(Dataset):
    def __init__(self, items ,bs=1, shuffle=False, sampler=None, batch_sampler=None, drop_last=False,
                 sampler_cls=None, batch_sampler_cls=BatchSampler, collate_fn=default_collate):
        super().__init__(items,collate_fn)
        self.sampler = batch_sampler
        self.rng,self.nw,self.offs = random.Random(),1,0
        self._delegate_items("get_batches","get_batch","collate")
        if self.sampler: return
        if not sampler: sampler = ifnone(sampler_cls, (SequentialSampler,RandomSampler)[shuffle])(items)
        self.sampler = batch_sampler_cls(sampler, bs, drop_last)

    def __iter__(self):
        torch.manual_seed(self.rng.randint(0,sys.maxsize))
        samps = list(enumerate(self.sampler))
        idxs = (b for i,b in samps if i%self.nw==self.offs)
        return self.get_batches(idxs)
    
    def get_batch(self, b): return [self.items[j] for j in b]
    def get_batches(self, idxs): return map(self.get_batch, idxs)
    def wif(self) : self.sampler.sampler = copy(self.sampler.sampler)

In [ ]:
def get_dl(bs=1, collate_fn=default_collate, num_workers=0, **kwargs):
    return DataLoader(testds, num_workers=num_workers, bs=bs, collate_fn=collate_fn, **kwargs)

In [ ]:
dl = get_dl(bs=4, num_workers=0)
t = twoepochs(dl)
test_eq(t, 'abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz')
test_eq(len(set(t)), 27)

In [ ]:
dl = get_dl(bs=4, num_workers=4, shuffle=True)
t = twoepochs(dl)
test_ne(t, 'abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz')
test_eq(len(set(t)), 27)
t

In [ ]:
# class _NextFetcher:
#     def __init__(self, dataset): self.dataset_iter = iter(dataset)
#     def fetch(self, possibly_batched_index): return next(self.dataset_iter)
# def create_fetcher(kind, dataset, auto_collation, collate_fn, drop_last): return _NextFetcher(dataset)
# _DatasetKind.create_fetcher = create_fetcher

#     def _delegate_items(self, *attrs):
#         for attr in attrs:
#             if hasattr(self.items,attr): setattr(self, attr, getattr(self.items, attr))
# 
#     def batch(self, s):
#         if self.done:
#             self.done=False
#             raise StopIteration
#         res = []
#         try:
#             for _ in range(self.bs): res.append(self.item(s))
#         except StopIteration:
#             if res==[]: raise StopIteration
#             self.done=True
#         return res

## BatchDS

In [ ]:
#export
class BaseDS(GetAttr):
    _xtra = ['show', 'decode', 'show_at', 'decode_at', 'decode_batch']
    def __init__(self, ds):
        self.default = self.ds = ds
        ds.wrapper = self
        self._delegate_ds("reset")

    def _delegate_ds(self, attr):
        if hasattr(self.ds,attr): setattr(self, attr, getattr(self.ds, attr))
            
    def reset(self): pass

In [ ]:
#export
class BatchDS(BaseDS, IterableDataset):
    _xtra = ['show', 'decode', 'show_at', 'decode_at', 'decode_batch']
    def __init__(self, ds ,bs=1, shuffle=False, sampler=None, batch_sampler=None, drop_last=False,
                 collate_fn=default_collate, sampler_cls=None, batch_sampler_cls=BatchSampler):
        self.default,self.ds,self.samp,self.collate_fn = ds,ds,batch_sampler,collate_fn
        self.rng,self.nw,self.offs,self.is_iterable = random.Random(),1,0,True
        for o in ("get_batches","get_batch","collate"): self._delegate_ds(o)
        if self.samp: return
        if not sampler: sampler = ifnone(sampler_cls, (SequentialSampler,RandomSampler)[shuffle])(ds)
        self.samp = batch_sampler_cls(sampler, bs, drop_last)

    def __iter__(self):
        torch.manual_seed(self.rng.randint(0,sys.maxsize))
        samps = list(enumerate(self.samp))
        idxs = (b for i,b in samps if i%self.nw==self.offs)
        return self.get_batches(idxs)
    
    def get_batch(self, b): return [self.ds[j] for j in b]
    def get_batches(self, idxs): return map(self.get_batch, idxs)
    def collate(self, idxs): return self.collate_fn(self.get_batches(idxs))
    def __len__(self): return len(self.samp)

In [ ]:
#export
def _wif(worker_id):
    info = get_worker_info()
    ds = info.dataset
    ds.nw,ds.offs = info.num_workers,info.id
    ds.samp.sampler = copy(ds.samp.sampler)

In [ ]:
class SleepyDS():
    def __init__(self,coll): self.coll=coll
    def __len__(self): return len(self.coll)
    def __getitem__(self,i): time.sleep(0.02); return self.coll[i]

In [ ]:
letters = list(string.ascii_lowercase)
def twoepochs(d): print(' '.join(''.join(o) for _ in range(2) for o in d))
bs = 4

In [ ]:
#export
def dataloader(ds, bs=1, num_workers=0, collate_fn=default_collate, **kwargs):
    if not isinstance(ds, IterableDataset): ds = BatchDS(ds, bs, **kwargs)
    return DataLoader(ds, num_workers=num_workers, batch_size=None,
                      worker_init_fn=_wif, collate_fn=noop)

In [ ]:
def get_dl(bs=1, collate_fn=default_collate, num_workers=0, **kwargs):
    return dataloader(SleepyDS(string.ascii_lowercase), bs, num_workers, collate_fn=collate_fn, **kwargs)

In [ ]:
dl = get_dl(bs=4, num_workers=4, shuffle=True)
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

gdxl ubiy znwj ecra spkh mtfq vo gdxl ubiy znwj ecra spkh mtfq vo


In [ ]:
dl = get_dl(bs=4, num_workers=4)
%time twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz
CPU times: user 21.1 ms, sys: 46.4 ms, total: 67.5 ms
Wall time: 396 ms


In [ ]:
dl = get_dl(bs=4, num_workers=0)
%time twoepochs(dl)
len(dl)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz
CPU times: user 5.71 ms, sys: 0 ns, total: 5.71 ms
Wall time: 1.05 s


7

In [ ]:
dl = get_dl(bs=4, num_workers=4, drop_last=True)
twoepochs(dl)
len(dl)

abcd efgh ijkl mnop qrst uvwx abcd efgh ijkl mnop qrst uvwx


6

In [ ]:
dl = get_dl(bs=4, num_workers=0, shuffle=True)
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

lahp kzyn rgmb sfdt xvco ueij wq svid tckw phjz raeu gfqy mlnb xo


In [ ]:
ds = SleepyDS(string.ascii_lowercase)
dl = get_dl(bs=4, num_workers=4, sampler=SequentialSampler(ds))
twoepochs(dl)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz


In [ ]:
dl = get_dl(num_workers=4, batch_sampler=BatchSampler(RandomSampler(ds), 8, False))
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

ypojsdrz hkvwnilc xqmbfgtu ae ypojsdrz hkvwnilc xqmbfgtu ae


In [ ]:
def rev_collate(s): return default_collate(list(reversed(s)))
dl = get_dl(bs=4, num_workers=4, collate_fn=rev_collate)
twoepochs(dl)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz


In [ ]:
class SleepyDS2(SleepyDS):
    def get_batch(self, b): return "".join([self[j] for j in b]) + '/'

dl = dataloader(SleepyDS2(string.ascii_lowercase), bs=4, num_workers=4)
twoepochs(dl)

abcd/ efgh/ ijkl/ mnop/ qrst/ uvwx/ yz/ abcd/ efgh/ ijkl/ mnop/ qrst/ uvwx/ yz/
